In [1]:
from datasets import load_dataset
import openai, json
key6 = ''
openai.api_key = key6

In [ ]:
prompt_q = data[2]['description']
prompt_q

In [ ]:
def get_chatgpt_qa_response(prompt_text, temperature = 0.7, max_tokens=2048):
    messages = [{"role":"system", "content": "You are a helpful assistant that answers the question provided."},
                {"role":"user", "content": 'Provide me the Python3 codes for solving the question: ' + prompt_text}]
    response = openai.ChatCompletion.create(
                model = "gpt-4-0314",
                messages = messages,
                temperature = temperature,
                max_tokens = max_tokens,    
                )
    return response#['choices'][0]['message']['content']

In [ ]:
response = get_chatgpt_qa_response( prompt_text = chatgpt_python_codecontest[85]['question'] )

In [ ]:
response['choices'][0]['message']['content']

In [2]:
with open( 'results/gpt4_humaneval.jsonl', "r") as f:
    gpt4_HumanEval = [json.loads(x) for x in f.read().strip().split("\n") ]
len(gpt4_HumanEval)

164

In [3]:
len( gpt4_HumanEval[10]['gen_completion'])

1284

In [4]:
gpt4_HumanEval[54]

{'question': 'Provide me the Python3 codes for sovling the question: \n\ndef same_chars(s0: str, s1: str):\n    """\n    Check if two words have the same characters.\n    >>> same_chars(\'eabcdzzzz\', \'dddzzzzzzzddeddabc\')\n    True\n    >>> same_chars(\'abcd\', \'dddddddabc\')\n    True\n    >>> same_chars(\'dddddddabc\', \'abcd\')\n    True\n    >>> same_chars(\'eabcd\', \'dddddddabc\')\n    False\n    >>> same_chars(\'abcd\', \'dddddddabce\')\n    False\n    >>> same_chars(\'eabcdzzzz\', \'dddzzzzzzzddddabc\')\n    False\n    """\n',
 'gold_completion': '    return set(s0) == set(s1)\n',
 'gen_completion': "def same_chars(s0: str, s1: str):\n    # Use set to get unique characters and sorted to compare them\n    return sorted(set(s0)) == sorted(set(s1))\n\n# Test cases\nprint(same_chars('eabcdzzzz', 'dddzzzzzzzddeddabc'))  # True\nprint(same_chars('abcd', 'dddddddabc'))  # True\nprint(same_chars('dddddddabc', 'abcd'))  # True\nprint(same_chars('eabcd', 'dddddddabc'))  # False\nprin

In [5]:
len_all = [len(x['gold_completion']) for x in gpt4_HumanEval ]
sum(len_all)/len(len_all)

180.8658536585366

In [6]:
max(len_all)

864

In [7]:
len_all.sort()

In [8]:
len_all[-3:]

[633, 699, 864]

In [9]:
len_all = [len(x['gen_completion']) for x in gpt4_HumanEval ]
sum(len_all)/len(len_all)

669.1951219512196

In [ ]:
from utils import parse_code_snippet

for i,instance in enumerate( gpt4_HumanEval):
    #pasrsed_codes = parse_code_snippet( prompt=instance['question'], raw_o=instance['gen_completion'] )
    #gpt4_HumanEval[i]['pasrsed_codes'] = pasrsed_codes
    pasrsed_codes = instance['pasrsed_codes']
    if pasrsed_codes.startswith('# CANNOT'):
        print(i+1)

In [10]:
gpt4_HumanEval[93]

{'question': 'Provide me the Python3 codes for sovling the question: \ndef encode(message):\n    """\n    Write a function that takes a message, and encodes in such a \n    way that it swaps case of all letters, replaces all vowels in \n    the message with the letter that appears 2 places ahead of that \n    vowel in the english alphabet. \n    Assume only letters. \n    \n    Examples:\n    >>> encode(\'test\')\n    \'TGST\'\n    >>> encode(\'This is a message\')\n    \'tHKS KS C MGSSCGG\'\n    """\n',
 'gold_completion': '    vowels = "aeiouAEIOU"\n    vowels_replace = dict([(i, chr(ord(i) + 2)) for i in vowels])\n    message = message.swapcase()\n    return \'\'.join([vowels_replace[i] if i in vowels else i for i in message])\n',
 'gen_completion': 'Here is the Python3 code for the encode function:\n\ndef encode(message):\n    def swap_case(c):\n        if c.isupper():\n            return c.lower()\n        else:\n            return c.upper()\n\n    def swap_vowel(c):\n        vowe

In [11]:
outputs = []
for ins in gpt4_HumanEval:
    outputs.append(json.dumps(ins))

with open( 'results/chatgpt_humaneval.jsonl', "w") as f:
    f.write("\n".join(outputs) + "\n")

In [13]:
def get_gpt4_continue(prompt_text, temperature = 0.7, max_tokens=2048):
    messages = [{"role":"system", "content": "You are a helpful assistant that continues the following codes. Make sure to only return continuation codes. Do not return anything else."},
                {"role":"user", "content": 'The proceding code is provided as: ' + prompt_text}]
    response = openai.ChatCompletion.create(
                model = "gpt-4-0314",
                messages = messages,
                temperature = temperature,
                max_tokens = max_tokens,    
                )
    return response#['choices'][0]['message']['content']

In [12]:
parsed_code = gpt4_HumanEval[85]['pasrsed_codes']
parsed_code = parsed_code[ :int( 0.7*len(parsed_code)) ]
response = get_gpt4_continue( prompt_text = parsed_code )

NameError: name 'get_gpt4_continue' is not defined

In [ ]:
response['choices'][0]['message']['content']

In [ ]:
parsed_code

In [ ]:
HUMANEVAL_EOS = ["\nclass", "\ndef", "\n#", "\n@", "\nprint", "\nif"]
NON_CODE_EOS = ["<|endoftext|>", "\n```", "\n</s>", "<|endofmask|>"]
EOS = HUMANEVAL_EOS + NON_CODE_EOS

def find_gen_func_sig(prompt):
    func_sig = ""
    for x in prompt.splitlines():
        if x.startswith("def ") and x.endswith(":"):
            # always pick the last one, since there could pre-defined functions.
            func_sig = x
    return func_sig

def remove_eos(gen):
    min_index = 1000
    for eos in EOS:
        if eos in gen:
            min_index = min(min_index, gen.index(eos))
    return gen[:min_index]

In [ ]:
idx = 0
raw_o = chatgpt_python_codecontest[idx]['gen_completion'] # response['choices'][0]['message']['content'] #
prompt = chatgpt_python_codecontest[idx]['question']
if "```" in raw_o:
    gen = raw_o.split("```")[1].strip()
    if gen.startswith("python"):
        gen = gen[len("python") :].strip()
    if gen.startswith(prompt.strip()):
        suf = gen.split(prompt.strip())[-1]
        suf = remove_eos(suf)
        gen = prompt.strip() + suf
    elif find_gen_func_sig(prompt) == '':
        gen = gen
    elif find_gen_func_sig(prompt) in gen:
        # same function sign is in the prompt
        sig = find_gen_func_sig(prompt)
        pre, suf = gen.split(sig)[0], gen.split(sig)[-1]
        suf = remove_eos(suf)
        gen = pre + sig + suf
    else:
        gen = f"# CANNOT PARSE CODE SNIPPET\n{gen}"
else:
    # cannot really handle parse just dump to file and maybe process later.
    gen = f"# CANNOT PARSE\n{raw_o}"
gen


In [ ]:
gen = raw_o.split("```")[1].strip()

In [ ]:
def call_edit(prompt_text, temperature = 0.7, max_tokens=2048):

    response = openai.Edit.create( model = "text-davinci-edit-001",
                                   input = prompt_text,
                                   instruction="make minimal modifications to rewrite the codes and maintain the same functionality ",
                                   temperature=temperature,
                                    #max_tokens=max_tokens,
                                      )
    return response

In [ ]:
response = call_edit( prompt_text = chatgpt_python_codecontest[0]['gen_completion'] )
response

In [ ]:
idx = 0
chatgpt_python_codecontest[idx]['gen_completion']['choices'][0]['text'][len( chatgpt_python_codecontest[idx]['question'] ): ]

In [ ]:
int(0.7*537)

In [ ]:
with open( 'results/regen_gpt-4-0314_20_0.7.jsonl', "r") as f:
    regen_gpt4 = [json.loads(x) for x in f.read().strip().split("\n") ]
len(regen_gpt4)

In [ ]:
regen_gpt4[8]['human_gen_text']#['choices']

In [ ]:
with open( 'results/regen_gpt-3.5-turbo_10_0.5_2nd.jsonl', "r") as f:
    regen_gpt4_01 = [json.loads(x) for x in f.read().strip().split("\n") ]
len(regen_gpt4_01)

In [ ]:
with open( 'results/regen_gpt-3.5-turbo_10_0.5.jsonl', "r") as f:
    regen_gpt4_02 = [json.loads(x) for x in f.read().strip().split("\n") ]
len(regen_gpt4_02)

In [ ]:
new_outputs = []
for i,ins in enumerate(regen_gpt4_01):
    temp = ins
    temp2 = ins['human_gen_text']['choices'] + regen_gpt4_02[i]['human_gen_text']['choices']
    temp['human_gen_text']['choices'] = temp2[:]

    temp3 = ins['machine_gen_text']['choices'] + regen_gpt4_02[i]['machine_gen_text']['choices']
    temp['machine_gen_text']['choices'] = temp3[:]

    new_outputs.append(json.dumps( temp) )

In [ ]:
temp = []
for ins in new_outputs:
    temp.append( ins )
    with open( 'results/regen_gpt-3.5-turbo_20_0.5.jsonl', "w") as f:
        f.write("\n".join(temp) + "\n")